##Install Libraries

In [ ]:
!git --version

git version 2.17.1


In [ ]:
!pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
!pip install audiolazy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 121 kB 5.1 MB/s 


##Import Libraries

In [104]:
import numpy as np
import pandas as pd
import matplotlib
import scipy
import seaborn
import librosa
import audiolazy
import sklearn
import os
import glob
import shutil

import scipy.signal as sig
from audiolazy import lazy_lpc

##Data Preparation

In [105]:
# Assign dataset's path to variable
dtsets = '/content/drive/MyDrive/Dataset TA/NewDataPrimer'

emotions = ['bahagia', 'kecewa', 'marah', 'sedih']  # Type of emotion to be classified

dtsets1 = '/content/drive/MyDrive/DataSpeechs'

In [106]:
print(os.listdir(dtsets))
print(os.listdir(dtsets1))

['Bahagia', 'Kecewa', 'Sedih', 'Marah']
['Salman', 'Aka', 'Tasya', 'Syahri', 'Erzal', 'Sabda', 'Febby', 'Sarah', 'Citra', 'Raina']


In [9]:
# Make folder for each data emotions and only need run this one time
'''
os.mkdir('/content/drive/MyDrive/SplitDataSpeech/Bahagia')
os.mkdir('/content/drive/MyDrive/SplitDataSpeech/Kecewa')
os.mkdir('/content/drive/MyDrive/SplitDataSpeech/Sedih')
os.mkdir('/content/drive/MyDrive/SplitDataSpeech/Marah')
'''

In [117]:
'''
Because the dataset used has not been separated according to the type of emotion and the model used is Unsupervised Learning,
it is necessary to separate it according to the type of emotion first.
This part is the code to separate the dataset according to the type of emotion into each emotion's folder,
so that the labeling process will be easier.

Note: We only need to run this one time.
'''

'''
fol_bahagia = '/content/drive/MyDrive/SplitDataSpeech/Bahagia'
fol_kecewa = '/content/drive/MyDrive/SplitDataSpeech/Kecewa'
fol_marah = '/content/drive/MyDrive/SplitDataSpeech/Marah'
fol_sedih = '/content/drive/MyDrive/SplitDataSpeech/Sedih'

for foldername in sorted(os.listdir(dtsets1)):
  folder = os.path.join(dtsets1, foldername)
  #print(folder)
  i = 0
  for filename in sorted(os.listdir(folder)):
    #print(filename)
    file_path = os.path.join(folder, filename)
    #print(file_path)
    
    if i <= 9:
      shutil.copy2(file_path, fol_bahagia)
    elif i in range(10, 20):
      shutil.copy2(file_path, fol_marah)
    elif i in range(20, 30):
      shutil.copy2(file_path, fol_kecewa)
    else:
      shutil.copy2(file_path, fol_sedih)
    i += 1

'''

In [118]:
# Folder checking in our datasets
os.listdir('/content/drive/MyDrive/SplitDataSpeech')

['Bahagia', 'Kecewa', 'Sedih', 'Marah']

###EDA

###Data Augmentation

In [ ]:
def noise(signal):
  """
  Adding White Noise.
  """
  #signal, s_rate = lbr.load(wav, duration=2, sr=None)
  noise_amp = 0.01*np.random.uniform()*np.amax(signal)   # more noise reduce the value to 0.5
  signal = signal + noise_amp * np.random.normal(size=signal.shape[0])
  return signal

def stretch(signal, rate=0.8):
  """
  Streching the Sound. Note that this expands the dataset slightly
  """
  #signal, s_rate = lbr.load(wav, duration=2, sr=None)
  signal = librosa.effects.time_stretch(signal, rate)
  return signal

def speedNpitch(signal):
  """
  Speed and Pitch Tuning.
  """
  #signal, s_rate = lbr.load(wav, duration=2, sr=None)
  length_change = np.random.uniform(low=0.8, high = 1)
  speed_fac = 1.2  / length_change
  tmp = np.interp(np.arange(0, len(signal), speed_fac), np.arange(0, len(signal)), signal)
  minlen = min(signal.shape[0], tmp.shape[0])
  signal *= 0
  signal[0:minlen] = tmp[0:minlen]
  return signal

## Linear Predictive Coding (LPC)

In [ ]:
def LPC_feature(wav, order=8):
  # Define LPF Features variable
  lpc_features = []

  # Read speech signal
  signal, s_rate = librosa.load(wav, duration=2, sr=None)

  # Pre-Emphasis
  emphasized_signal = librosa.effects.preemphasis(signal, coef=0.97)

  # Frame Blocking and Windowing
  '''
  frames_win = np.abs(lbr.stft(emphasized_signal)) # using kind of hann window
  idx_frame = len(frames_win)//2
  frames_win = frames_win[idx_frame]
  '''

  # Autocorrelation
  autocorrelation = sig.fftconvolve(emphasized_signal, emphasized_signal[::-1])
  autocorr_coeff = autocorrelation[autocorrelation.size//2:][:(order + 1)]      # Autocorrelation Coefficients

  # LPC features
  lpc_coeff_ld = lazy_lpc.levinson_durbin(autocorr_coeff, 3)
  lpc_features = lpc_coeff_ld.numerator[1:]
  
  return lpc_features

In [ ]:
LPC_feature('/content/drive/MyDrive/Dataset TA/NewDataPrimer/Bahagia/Aka Bahagia 1.wav')

[-0.924572209945653, 0.7073411839646861, -0.37092528522518536]

In [ ]:
def load_dataset():
  sounds, emotions = [], []

  for dirname, _, filenames in os.walk(dtsets):
    for filename in filenames:
      #print(os.path.join(dirname, filename))
      sounds.append(os.path.join(dirname, filename))
      #print(filename)
      label = filename.split(' ')[-2]
      #print(label)
      emotions.append(label.lower())
  return {'Audio': sounds, 'Emotion': emotions}

In [ ]:
import time

start_time = time.time()
Trial_dict = load_dataset()
print("--- Data loaded. Loading time: %s seconds ---" % (time.time() - start_time))

--- Data loaded. Loading time: 0.0053293704986572266 seconds ---


In [ ]:
audio = Trial_dict['Audio']
label = Trial_dict['Emotion']

data = pd.DataFrame(label, columns = ['Emotion'])
data = pd.concat([data, pd.DataFrame(audio, columns = ['Path'])], axis=1)
data.head(-1)

,Emotion,Path
0,bahagia,/content/drive/MyDrive/Dataset TA/NewDataPrime...
1,bahagia,/content/drive/MyDrive/Dataset TA/NewDataPrime...
2,bahagia,/content/drive/MyDrive/Dataset TA/NewDataPrime...
3,bahagia,/content/drive/MyDrive/Dataset TA/NewDataPrime...
4,bahagia,/content/drive/MyDrive/Dataset TA/NewDataPrime...
...,...,...
74,marah,/content/drive/MyDrive/Dataset TA/NewDataPrime...
75,marah,/content/drive/MyDrive/Dataset TA/NewDataPrime...
76,marah,/content/drive/MyDrive/Dataset TA/NewDataPrime...
77,marah,/content/drive/MyDrive/Dataset TA/NewDataPrime...


##Features Engineering

## Next (Soon...)